In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
pd.set_option('display.max_rows', None,'display.max_columns', None)
import plotly.express as px

2022-04-20 15:00:07.788429: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/count
2022-04-20 15:00:07.788451: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/count
2022-04-20 15:00:07.788455: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/api
2022-04-20 15:00:07.788460: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/api


In [2]:
Hitters = pd.read_csv('/Users/lei/Desktop/Lecture_Notes/McGill_Courses/James_2021_Rscripts/Data/Hitters.csv', index_col='Unnamed: 0')
print(Hitters.columns)
print(Hitters.shape)
# Hitters.Salary.isna().sum() # count all the missing elements
# Hitters.isnull().sum()

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'Salary', 'NewLeague'],
      dtype='object')
(322, 20)


In [3]:
Gitters = Hitters.dropna()
print(Gitters.shape)
Gitters.head()

(263, 20)


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [4]:
pd_dums = pd.get_dummies(Gitters[['League', 'Division','NewLeague']])

Gitters_dum_all = pd.concat([Gitters.loc[:, ~Gitters.columns.isin(['League', 'Division','NewLeague'])], pd_dums], axis=1)
Gitters_dum = Gitters_dum_all.loc[:, ~Gitters_dum_all.columns.isin(['League_A', 'Division_E','NewLeague_A'])]
Gitters_dum.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_N,Division_W,NewLeague_N
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0,1,1,1
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0,0,1,0
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0,1,0,1
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5,1,0,1
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750.0,0,1,0


In [5]:
from sklearn.preprocessing import scale
Gitters_dum_sd = pd.DataFrame(scale(Gitters_dum.loc[:, ~Gitters_dum.columns.isin(['Salary'])]), columns=Gitters_dum.columns[~Gitters_dum.columns.isin(['Salary'])])
Gitters_dum_sd.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
0,-0.602900,-0.595675,-0.528551,-1.206112,-0.522063,-0.097527,1.397893,0.346791,0.174373,-0.002920,-0.121671,0.258966,0.435334,1.221499,-0.523191,0.213352,1.058758,0.981166,1.075052
1,0.512542,0.492260,0.729966,0.441515,0.794060,1.609373,-0.901200,-0.452865,-0.409892,-0.076054,-0.415105,-0.199590,0.010373,2.109109,-0.253863,0.819964,-0.944503,0.981166,-0.930187
2,0.628167,0.736490,0.958788,0.402286,1.026317,-0.189792,0.770868,1.301558,1.318174,1.898565,1.412051,1.572666,0.355654,-0.324661,-0.744179,-0.848219,1.058758,-1.019196,1.075052
3,-0.562092,-0.462459,-0.185319,-0.617673,-0.367225,-0.512719,-1.110209,-0.990935,-0.960153,-0.697693,-0.947521,-0.881228,-0.862315,1.840678,-0.543909,-0.696566,1.058758,-1.019196,1.075052
4,1.294712,1.358167,-0.871783,0.755349,-0.018840,-0.282057,0.770868,0.766993,0.634985,-0.612370,0.422846,0.017294,-0.251434,-0.031177,2.087225,2.488147,-0.944503,0.981166,-0.930187


In [6]:
np.random.seed(13)
testid = np.random.choice(len(Gitters), size=len(Gitters) // 3, replace=False)
testid

array([242, 159, 188,  98, 172, 247,  97, 158, 223, 198, 100, 234,   7,
       106,  30, 142, 183, 209, 195, 108, 110,  12, 162, 191, 114, 128,
       151,  67,  92,  51, 140,  80,  68,  23,  70, 115, 131, 163,  78,
        66,  36, 127,  63,  65,   9,  33, 222,  90, 224, 119, 125, 220,
       245, 168, 262, 118, 250, 155, 255, 199,  79, 101, 143,  64, 169,
        42, 248,  93, 193, 211, 210, 150, 252,  84,  39, 201, 170,  82,
        10, 148,  13,  32,  15, 206, 196, 228,  28])

In [7]:
# X_train, X_test, y_train, y_test = 
X_train = Gitters_dum.loc[:, ~Gitters_dum.columns.isin(['Salary'])].iloc[~testid, :]
y_train = Gitters_dum.Salary.iloc[~testid]
X_test = Gitters_dum.loc[:, ~Gitters_dum.columns.isin(['Salary'])].iloc[testid, :]
y_test = Gitters_dum.Salary.iloc[testid]
nrow, ncol = X_train.shape

In [8]:
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error

lfit = linear_model.LinearRegression().fit(X=X_train, y=y_train)
y_hat = lfit.predict(X=X_test)

mean_absolute_error(y_test, y_hat)


252.61134261488954

In [9]:
# Define Sequential model with 3 layers
modnn = keras.Sequential(
    [
        layers.Dense(50, activation="relu", input_shape=(X_train.shape[1],)), # hidden layer with relu activation 
        layers.Dropout(.4), # dropout regularization
        layers.Dense(1), # output layer
    ]
)

modnn.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])

modnn.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1000      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 1,051
Trainable params: 1,051
Non-trainable params: 0
_________________________________________________________________


2022-04-20 15:00:14.227998: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
modnn.fit(x=X_train, y=y_train, epochs=600, batch_size=32, validation_data=(X_test, y_test))


2022-04-20 15:00:14.600035: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:179] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/600
3/3 [==============================] - 1s 60ms/step - loss: 822164.9375 - mae: 537.1786 - val_loss: 300386.4375 - val_mae: 295.4972
Epoch 2/600
3/3 [==============================] - 0s 7ms/step - loss: 570311.4375 - mae: 466.0150 - val_loss: 210549.8438 - val_mae: 248.5888
Epoch 3/600
3/3 [==============================] - 0s 7ms/step - loss: 473860.9688 - mae: 454.4682 - val_loss: 186931.7188 - val_mae: 238.6125
Epoch 4/600
3/3 [==============================] - 0s 7ms/step - loss: 519055.6250 - mae: 465.3689 - val_loss: 182580.2188 - val_mae: 235.4124
Epoch 5/600
3/3 [==============================] - 0s 7ms/step - loss: 591973.6250 - mae: 496.2587 - val_loss: 163410.5625 - val_mae: 232.8883
Epoch 6/600
3/3 [==============================] - 0s 7ms/step - loss: 391052.0000 - mae: 425.2636 - val_loss: 161793.3438 - val_mae: 228.3348
Epoch 7/600
3/3 [==============================] - 0s 7ms/step - loss: 432088.7500 - mae: 455.0908 - val_loss: 164712.7656 - val_mae: 226.237

In [11]:

# tf.keras.utils.plot_model(modnn, show_shapes=True)
print(modnn.history.history.keys())
# modnn.history.epoch

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])


In [12]:
modnn_res = pd.DataFrame.from_dict(modnn.history.history)
modnn_res.head()

,loss,mae,val_loss,val_mae
0,822164.93750,537.178589,300386.43750,295.497192
1,570311.43750,466.015045,210549.84375,248.588760
2,473860.96875,454.468201,186931.71875,238.612534
3,519055.62500,465.368896,182580.21875,235.412384
4,591973.62500,496.258698,163410.56250,232.888306


In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=('Loss', 'MAE'))

# fig = go.Figure()
fig.add_trace(go.Scatter(x=modnn.history.epoch, y=modnn_res.loss, name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(x=modnn.history.epoch, y=modnn_res.val_loss, name='val_loss'), row=1, col=1)

fig.add_trace(go.Scatter(x=modnn.history.epoch, y=modnn_res.mae, name='mae'), row=1, col=2)
fig.add_trace(go.Scatter(x=modnn.history.epoch, y=modnn_res.val_mae, name='val_mae'), row=1, col=2)

fig.show()